# Fixing the SN data frame

In [1]:
import load_data
import preprocess_spectra
import pandas
from importlib import reload
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
cmap = get_cmap('viridis')
import numpy
import specplotlib
from sklearn.linear_model import LinearRegression
%matplotlib inline

In [2]:
SN_df_w_salt_w_class = pandas.read_csv('good_df.csv')
SN_df_w_salt_w_class.head()

,idx,SN,zhel_spec,tmax(B)_spec,tmax(B)_err_spec,ref_spec,Dm15_spec,Dm15_err_spec,ref1_spec,M_B_spec,...,z_cmb_salt,z_err_salt,m_B_salt,m_B_err_salt,s_salt,s_err_salt,c_salt,c_err_salt,mu_salt,mu_err_salt
0,0,1994D,0.0029,49432.5,0.1,"B,M,SA,SN",1.37,0.03,B,-99.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1994M,0.0232,49474.5,0.9,"B,M,SA,SN",1.26,0.06,B,-19.21,...,0.024,0.001,16.30,0.159,0.78,0.02,0.11,0.02,35.207,0.205
2,2,1994S,0.0151,49518.3,0.3,"B,M,SA,SN",0.94,0.06,B,-19.55,...,0.015,0.001,14.77,0.196,1.09,0.05,-0.01,0.02,34.372,0.280
3,3,1994T,0.0346,49514.4,0.4,"M,SN",1.51,0.08,SN,-99.99,...,0.036,0.001,17.05,0.084,0.84,0.04,0.02,0.05,36.264,0.181
4,4,1994ae,0.0043,49685.5,0.8,"B,M,SA,SN",0.96,0.04,B,-99.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Add constitution to df

SN_df_w_salt_w_class = pandas.read_csv('good_df.csv')
SN_df_w_salt_w_class = SN_df_w_salt_w_class.set_index("SN", drop=False)
#SN_df_w_salt_w_class.index = [s.lower() for s in SN_df_w_salt_w_class.index]

MLCS31 = pandas.read_csv('MLCS31.txt', delimiter='\s+', header=None, skiprows=5)
MLCS31.columns = ["SNIa_ml31", "z_ml31", "z_err_ml31", "x1_ml31", "x1_err_ml31", "c_ml31", "c_err_ml31", "mu_ml31", "mu_err_ml31"]
MLCS31 = MLCS31.set_index('SNIa_ml31', drop=False)
MLCS31.index = [s.lower() for s in MLCS31.index]

index_tmp = MLCS31.index
index_new = []
for s in index_tmp:
    if 'sn' in s:
        s = s[2:]
        if (s[0] == '0') or (s[0] == '1'):
            s = '20' + s
        else:
            s = '19' + s
    index_new += [s]
MLCS31.index = index_new

SN_df_MLCS31 = SN_df_w_salt_w_class.merge(MLCS31, left_index=True, right_index=True, how='left')
SN_df_MLCS31.head()


,idx,SN,zhel_spec,tmax(B)_spec,tmax(B)_err_spec,ref_spec,Dm15_spec,Dm15_err_spec,ref1_spec,M_B_spec,...,mu_err_salt,SNIa_ml31,z_ml31,z_err_ml31,x1_ml31,x1_err_ml31,c_ml31,c_err_ml31,mu_ml31,mu_err_ml31
SN,,,,,,,,,,,,,,,,,,,,,
1994D,0,1994D,0.0029,49432.5,0.1,"B,M,SA,SN",1.37,0.03,B,-99.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1994M,1,1994M,0.0232,49474.5,0.9,"B,M,SA,SN",1.26,0.06,B,-19.21,...,0.205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1994S,2,1994S,0.0151,49518.3,0.3,"B,M,SA,SN",0.94,0.06,B,-19.55,...,0.280,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1994T,3,1994T,0.0346,49514.4,0.4,"M,SN",1.51,0.08,SN,-99.99,...,0.181,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1994ae,4,1994ae,0.0043,49685.5,0.8,"B,M,SA,SN",0.96,0.04,B,-99.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
SN_df_MLCS31.shape, SN_df_MLCS31['mu_ml31'].values, numpy.isfinite(SN_df_MLCS31['mu_ml31'].values).sum()

((229, 56),
 array([   nan,    nan,    nan,    nan,    nan,    nan,    nan, 36.535,
           nan,    nan,    nan,    nan, 38.887,    nan,    nan, 36.059,
        34.004,    nan,    nan,    nan,    nan,    nan, 33.56 ,    nan,
           nan, 35.165,    nan, 33.269,    nan, 34.372, 34.449,    nan,
           nan, 36.885,    nan, 34.17 , 35.305,    nan, 34.462,    nan,
           nan,    nan, 35.811,    nan, 33.913, 33.662, 36.666, 34.404,
        34.317, 34.526,    nan, 35.61 ,    nan, 36.33 ,    nan,    nan,
           nan,    nan, 34.375,    nan,    nan, 34.925,    nan,    nan,
           nan,    nan,    nan, 36.472, 34.044,    nan,    nan,    nan,
        35.003, 34.562, 36.295, 34.249, 33.795,    nan, 34.124, 33.728,
           nan,    nan,    nan, 35.433,    nan,    nan,    nan, 35.771,
        33.506,    nan,    nan,    nan, 35.631, 33.127,    nan, 34.305,
        33.505,    nan,    nan, 34.415,    nan, 34.209,    nan, 34.032,
        35.82 , 35.286, 36.289, 34.037,    nan, 35.2

## checks

In [20]:
SN_df_w_salt_w_class.columns

Index(['idx', 'SN', 'zhel_spec', 'tmax(B)_spec', 'tmax(B)_err_spec',
       'ref_spec', 'Dm15_spec', 'Dm15_err_spec', 'ref1_spec', 'M_B_spec',
       'M_B_err_spec', 'B-V_spec', 'B-V_err_spec', 'Bm-Vm_spec',
       'Bm-Vm_err_spec', 'Phot_ref_spec', 'ref..2', 'n_spec', 'RA', 'DEC',
       'z_CMB', 'z_CMB_salt2_err', 'm_B_salt2', 'm_B_salt2_err', 'x_1_salt2',
       'x_1_salt2_err', 'c_salt2', 'c_salt2_err', 'mu_salt2', 'mu_salt2_err',
       'EW5972_spec', 'EW6355_spec', 'v6355_spec', 'Phase_spec', 'BClass_spec',
       'WClass_spec', 'SNIa_salt', 'z_cmb_salt', 'z_err_salt', 'm_B_salt',
       'm_B_err_salt', 's_salt', 's_err_salt', 'c_salt', 'c_err_salt',
       'mu_salt', 'mu_err_salt'],
      dtype='object')

In [18]:
MLCS31.head(3)

,SNIa_ml31,z_ml31,z_err_ml31,x1_ml31,x1_err_ml31,c_ml31,c_err_ml31,mu_ml31,mu_err_ml31
03d1aw,03D1aw,0.582,0.001,-0.262,0.141,0.144,0.113,43.098,0.207
03d1ax,03D1ax,0.496,0.001,0.178,0.118,0.117,0.115,42.322,0.194
03d1bp,03D1bp,0.346,0.001,0.157,0.121,0.268,0.224,41.516,0.255


In [44]:
SN_df_w_salt_w_class.head(3)

,idx,SN,zhel_spec,tmax(B)_spec,tmax(B)_err_spec,ref_spec,Dm15_spec,Dm15_err_spec,ref1_spec,M_B_spec,...,z_cmb_salt,z_err_salt,m_B_salt,m_B_err_salt,s_salt,s_err_salt,c_salt,c_err_salt,mu_salt,mu_err_salt
idx,,,,,,,,,,,,,,,,,,,,,
0,0,1994D,0.0029,49432.5,0.1,"B,M,SA,SN",1.37,0.03,B,-99.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1994M,0.0232,49474.5,0.9,"B,M,SA,SN",1.26,0.06,B,-19.21,...,0.024,0.001,16.30,0.159,0.78,0.02,0.11,0.02,35.207,0.205
2,2,1994S,0.0151,49518.3,0.3,"B,M,SA,SN",0.94,0.06,B,-19.55,...,0.015,0.001,14.77,0.196,1.09,0.05,-0.01,0.02,34.372,0.280


In [45]:
print('len const_salt: %s' % len(MLCS31['SNIa_salt'].unique()))
print('len SN_salt_df In SN_df: %s' % (len(SN_df_MLCS31)))
print('len const_salt In SN_df: %s' % numpy.isfinite(SN_df_MLCS31['s_salt'].values).sum())
print('len spectra data in SN_df: %s' % (SN_df_MLCS31['M_B_spec'].values>-75).sum())

len const_salt: 397
len SN_salt_df In SN_df: 229
len const_salt In SN_df: 92
len spectra data in SN_df: 103


In [16]:
SN_df_MLCS31['SNIa_ml31'].unique()

array([nan, 'sn95ac', 'sn96ab', 'sn96bl', 'sn96bo', 'sn97do', 'sn98ab',
       'sn98bp', 'sn98co', 'sn98de', 'sn98dx', 'sn98ef', 'sn98eg',
       'sn99aa', 'sn99cc', 'sn99da', 'sn99dq', 'sn99ef', 'sn99ej',
       'sn99ek', 'sn99gd', 'sn99gp', 'sn00cf', 'sn00dk', 'sn00fa',
       'sn01az', 'sn01bf', 'sn01cp', 'sn01da', 'sn01eh', 'sn01en',
       'sn01ep', 'sn01fe', 'sn01fh', 'sn02bf', 'sn02ck', 'sn02cr',
       'sn02de', 'sn02dj', 'sn02do', 'sn02dp', 'sn02es', 'sn02fb',
       'sn02ha', 'sn02hd', 'sn02he', 'sn02hu', 'sn02hw', 'sn02jy',
       'sn02kf', 'sn03ch', 'sn03cq', 'sn03fa', 'sn03hu', 'sn03ic',
       'sn03it', 'sn03iv', 'sn04as', 'sn04bg', 'sn05el', 'sn05eq',
       'sn05eu', 'sn05hc', 'sn05iq', 'sn05kc', 'sn05ki', 'sn05lz',
       'sn05mc', 'sn05na', 'sn06ac', 'sn06al', 'sn06ax', 'sn06az',
       'sn06bq', 'sn06bt', 'sn06bu', 'sn06bw', 'sn06bz', 'sn06cf',
       'sn06cj', 'sn06cp', 'sn06cq', 'sn06em', 'sn06et', 'sn06gj',
       'sn06gr', 'sn06gt', 'sn06hb', 'sn06kf', 'sn06le', 

In [ ]:
############## still missing all the ones in the middle and how to get all the const in - see:
# https://archive.stsci.edu/prepds/ps1cosmo/scolnic_datatable.html 
# https://arxiv.org/pdf/1611.07042.pdf
# https://arxiv.org/pdf/1710.00845.pdf

In [13]:
SN_df_MLCS31.to_csv('SNdf_with_MLCS31.csv', index=False)